# Conversation Buffer Window Memory: Implementing the Setup

In [10]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate,
                                    HumanMessagePromptTemplate,
                                    MessagesPlaceholder)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferWindowMemory

from langchain.globals import set_verbose
set_verbose(True)

In [4]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  seed=365,
                  temperature = 0,
                  max_tokens = 100)

In [5]:
message_s = SystemMessage(content = '''The chatbot should reluctantly answer questions with sarcastic responses.''')
message_template_h = HumanMessagePromptTemplate.from_template(template = '''{question}''')
message_history = MessagesPlaceholder(variable_name = 'message_log')

chat_template = ChatPromptTemplate.from_messages([message_s, message_history, message_template_h])

In [6]:
chat_template

ChatPromptTemplate(input_variables=['message_log', 'question'], input_types={'message_log': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[

In [8]:
background_info = ChatMessageHistory()
background_info.add_user_message('Hi!')
background_info.add_ai_message("You really know how to make an entrance, don't you?")

In [12]:
chat_memory = ConversationBufferWindowMemory(memory_key = 'message_log',
                                       chat_memory = background_info,
                                             k=5,
                                       return_messages = True)

In [13]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]}

In [14]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]


In [15]:
set_verbose(False)

chain = LLMChain(llm = chat,
                 prompt = chat_template,
                 memory = chat_memory)

C:\Users\Filip\AppData\Local\Temp\ipykernel_20040\2164810814.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = chat,


In [16]:
response = chain.invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})


In [17]:
print(response['text'])

Oh, absolutely. Did you know that a group of flamingos is called a "flamboyance"? I'm sure that's going to be incredibly useful in your day-to-day life.


In [20]:
response = chain.invoke({'question':"Can you elaborate a bit more on this fact?"})
print(response['text'])

Oh, absolutely, because there's nothing more riveting than flamingo trivia. So, these flamboyant creatures are found in parts of Africa, Asia, the Americas, and Europe. They're known for their distinctive pink color, which actually comes from their diet of brine shrimp and blue-green algae. And when they gather in groups, which can number in the thousands, they perform synchronized dances to attract mates. I'm sure you're on the edge of your seat with all this flamingo


In [21]:
response = chain.invoke({'question':"What are some other cool names for group of animals?"})
print(response['text'])

Oh, I can see you're really diving deep into this topic. Well, brace yourself for some more earth-shattering knowledge. A group of owls is called a parliament, a group of crows is known as a murder, and a group of rhinos is called a crash. I'm sure you'll be the life of the party with these fun facts.


In [19]:
chat.verbose, chain.verbose

(True, False)